In [1]:
import numpy as np
import casadi as ca

In [2]:
def flux_balance_analysis():
    """
    Performs Flux Balance Analysis (FBA) on a simple toy metabolic network.

    The network consists of two metabolites (A, B) and four reactions (v1, v2, v3, v4).
    v1: -> A (uptake of A)
    v2: A -> B
    v3: B -> A
    v4: B -> (excretion of B, considered as biomass)

    The goal is to maximize the biomass reaction flux (v4).
    """
    # 1. Define the Stoichiometric Matrix (S)
    # The matrix describes the relationship between metabolites and reactions.
    # Rows correspond to metabolites (A, B)
    # Columns correspond to reactions (v1, v2, v3, v4)
    # S_ij is the stoichiometric coefficient of metabolite i in reaction j.
    # A: +v1 -v2 +v3
    # B: +v2 -v3 -v4
    S = np.array([
        [1, -1, 1, 0],   # Metabolite A
        [0, 1, -1, -1]    # Metabolite B
    ])
    
    # 2. Define the number of metabolites and reactions
    num_metabolites, num_reactions = S.shape
    print(f"Number of metabolites: {num_metabolites}")
    print(f"Number of reactions: {num_reactions}")
    print("\nStoichiometric Matrix (S):")
    print(S)

    # 3. Set up the Optimization Problem using CasADi
    # Create a CasADi symbol for the flux vector 'v'
    v = ca.SX.sym('v', num_reactions)

    # 4. Define the Objective Function
    # We want to maximize the flux of the biomass reaction, which is v4.
    # In CasADi's solvers, optimization is typically minimization, so we minimize -v4.
    # The index for v4 is 3 (0-indexed).
    objective = v[3]

    # 5. Define the Constraints
    # a) Steady-state constraint: S * v = 0
    # This means the production rate of each metabolite equals its consumption rate.
    steady_state_constraint = ca.mtimes(S, v)

    # b) Flux Bounds (Lower and Upper bounds for each reaction flux)
    # Let's assume the following bounds:
    # v1 (uptake of A): 0 <= v1 <= 10
    # v2 (A -> B): 0 <= v2 <= inf (irreversible)
    # v3 (B -> A): 0 <= v3 <= inf (irreversible)
    # v4 (biomass): 0 <= v4 <= inf
    lb = np.array([10, 0, 0, 0])
    ub = np.array([10, np.inf, np.inf, np.inf])

    # 6. Formulate the Linear Programming (LP) Problem
    nlp = {'x': v, 'f': -objective, 'g': steady_state_constraint}

    # 7. Create a Solver
    # We will use 'qpoases' which is suitable for quadratic and linear programs.
    # For a pure LP, 'glpk' or 'ipopt' could also be used.
    solver_options = {'ipopt': {'print_level': 0}, 'print_time': 0}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_options)

    # 8. Solve the LP Problem
    # We need to provide bounds for the decision variables ('x', our fluxes 'v')
    # and the constraints ('g', our steady-state S*v=0).
    # Since S*v must be exactly 0, the lower and upper bounds for 'g' are both 0.
    
    # Define an initial guess for the fluxes (x0). This can help in finding
    # alternative optimal solutions in non-strictly convex problems.
    delta = 1000
    initial_guess = np.array([10, 10+delta, delta, 10])
    
    solution = solver(x0=initial_guess, lbx=lb, ubx=ub, lbg=0, ubg=0)

    # 9. Extract and Print the Results
    if solver.stats()['success']:
        optimal_fluxes = solution['x']
        biomass_flux = optimal_fluxes[3]

        print("\n--- FBA Results ---")
        print(f"Optimization was successful.")
        print(f"Maximum biomass flux (v4): {biomass_flux}")
        print("\nOptimal flux distribution:")
        for i in range(num_reactions):
            print(f"  v{i+1}: {optimal_fluxes[i]}")
    else:
        print("\n--- FBA Results ---")
        print("Optimization failed.")
        print("Solver statistics:", solver.stats())


if __name__ == "__main__":
    flux_balance_analysis()

Number of metabolites: 2
Number of reactions: 4

Stoichiometric Matrix (S):
[[ 1 -1  1  0]
 [ 0  1 -1 -1]]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************


--- FBA Results ---
Optimization was successful.
Maximum biomass flux (v4): 10

Optimal flux distribution:
  v1: 10
  v2: 2913.53
  v3: 2903.53
  v4: 10


In [3]:
import numpy as np
import casadi as ca

def pheflux_analysis():
    """
    Performs Pheflux analysis on a simple toy metabolic network.

    Pheflux uses the same steady-state constraints as FBA, but its objective
    is to find a flux distribution (v) that is most consistent with a given
    gene expression profile (g).

    The network consists of two metabolites (A, B) and four reactions (v1, v2, v3, v4).
    v1: -> A (uptake of A)
    v2: A -> B
    v3: B -> A
    v4: B -> (excretion of B, considered as biomass)

    The objective function to minimize is a form of Kullback-Leibler divergence:
    sum_i (v_i/V) * log((v_i/V) / (g_i/G))
    where V = sum(v_i) and G = sum(g_i).
    """
    # 1. Define the Stoichiometric Matrix (S)
    # The matrix describes the relationship between metabolites and reactions.
    # Rows correspond to metabolites (A, B)
    # Columns correspond to reactions (v1, v2, v3, v4)
    # A: +v1 -v2 +v3
    # B: +v2 -v3 -v4
    S = np.array([
        [1, -1, 1, 0],   # Metabolite A
        [0, 1, -1, -1]    # Metabolite B
    ])
    
    # 2. Define the number of metabolites and reactions
    num_metabolites, num_reactions = S.shape
    print(f"Number of metabolites: {num_metabolites}")
    print(f"Number of reactions: {num_reactions}")
    print("\nStoichiometric Matrix (S):")
    print(S)

    # 3. Set up the Optimization Problem using CasADi
    # Create a CasADi symbol for the flux vector 'v'
    v = ca.SX.sym('v', num_reactions)

    # 4. Define the Objective Function for Pheflux
    # Gene expression data (g) and its sum (G)
    g = np.array([1.0, 1.0, 1.0, 1.0])
    G = np.sum(g)
    
    # Small epsilon for numerical stability to avoid log(0) or division by zero
    epsilon = 1e-9
    
    # Total flux V
    V = ca.sum1(v)
    
    # The objective is to minimize the KL-divergence between the flux
    # proportions (v/V) and the gene expression proportions (g/G).
    v_proportions = v / (V + epsilon)
    g_proportions = g / G
    
    # We add epsilon to the argument of the log as well
    objective = -ca.sum1(v_proportions * ca.log(v_proportions + epsilon / g_proportions))

    # 5. Define the Constraints
    # a) Steady-state constraint: S * v = 0
    steady_state_constraint = ca.mtimes(S, v)

    # b) Flux Bounds (Lower and Upper bounds for each reaction flux)
    # Let's assume the following bounds:
    # v1 (uptake of A): 0 <= v1 <= 10
    # v2 (A -> B): 0 <= v2 <= inf (irreversible)
    # v3 (B -> A): 0 <= v3 <= inf (irreversible)
    # v4 (biomass): 0 <= v4 <= inf
    lb = np.array([10, 0, 0, 0])
    ub = np.array([10, np.inf, np.inf, np.inf])

    # 6. Formulate the Nonlinear Programming (NLP) Problem
    nlp = {'x': v, 'f': -objective, 'g': steady_state_constraint}

    # 7. Create a Solver
    # Since the objective is nonlinear (due to the log), we use a solver like 'ipopt'.
    solver_options = {'ipopt': {'print_level': 0}, 'print_time': 0}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_options)

    # 8. Solve the NLP Problem
    # We need to provide bounds for the decision variables ('x', our fluxes 'v')
    # and the constraints ('g', our steady-state S*v=0).
    # Since S*v must be exactly 0, the lower and upper bounds for 'g' are both 0.
    
    # Define an initial guess for the fluxes (x0). This is important for NLPs.
    initial_guess = np.array([1.0, 1.0, 0.0, 1.0])
    
    solution = solver(x0=initial_guess, lbx=lb, ubx=ub, lbg=0, ubg=0)

    # 9. Extract and Print the Results
    if solver.stats()['success']:
        optimal_fluxes = solution['x']
        biomass_flux = optimal_fluxes[3]

        print("\n--- Pheflux Results ---")
        print(f"Optimization was successful.")
        print(f"Predicted biomass flux (v4): {biomass_flux}")
        print("\nOptimal flux distribution:")
        for i in range(num_reactions):
            print(f"  v{i+1}: {optimal_fluxes[i]}")
    else:
        print("\n--- Pheflux Results ---")
        print("Optimization failed.")
        print("Solver statistics:", solver.stats())


if __name__ == "__main__":
    pheflux_analysis()


Number of metabolites: 2
Number of reactions: 4

Stoichiometric Matrix (S):
[[ 1 -1  1  0]
 [ 0  1 -1 -1]]

--- Pheflux Results ---
Optimization was successful.
Predicted biomass flux (v4): 10

Optimal flux distribution:
  v1: 10
  v2: 18.1917
  v3: 8.19172
  v4: 10


In [4]:
import numpy as np
import casadi as ca

def reversible_maxent():
    """
    Performs a flux analysis based on maximizing a metabolic entropy function.

    This method splits each net flux v_i into forward (vf_i) and reverse (vr_i)
    components, where v_i = vf_i - vr_i.

    The network consists of two metabolites (A, B) and four reactions.
    The objective function to minimize is:
    -sum(vf_i * (log(vf_i) - log(g_i) ) + vr_i * (log(vr_i) - log(g_i) ) )

    This is equivalent to maximizing the entropy-like term.
    The problem is subject to steady-state constraints (S*v = 0) and a
    fixed uptake flux for reaction 1 (v1 = 10).
    """
    # 1. Define the Stoichiometric Matrix (S)
    # Rows correspond to metabolites (A, B)
    # Columns correspond to reactions (v1, v2, v3, v4)
    S = np.array([
        [1, -1, 1,  0],   # Metabolite A
        [0,  1,-1, -1]    # Metabolite B
    ])
    
    # 2. Define model parameters
    num_metabolites, num_reactions = S.shape
    print(f"Number of metabolites: {num_metabolites}")
    print(f"Number of reactions: {num_reactions}")
    print("\nStoichiometric Matrix (S):")
    print(S)

    # 3. Set up the Optimization Problem using CasADi
    # We define separate variables for forward (vf) and reverse (vr) fluxes.
    vf = ca.SX.sym('vf', num_reactions)
    vr = ca.SX.sym('vr', num_reactions)
    
    # The full vector of optimization variables
    x = ca.vertcat(vf, vr)

    # 4. Define the Objective Function
    # Gene expression data (g)
    g = np.array([10, 10, 10, 10])
    
    # Small epsilon for numerical stability to avoid log(0)
    epsilon = 1e-6
    
    # Entropy terms for forward and reverse fluxes
    forward_entropy = vf * (ca.log(vf + epsilon) - ca.log(g))
    reverse_entropy = vr * (ca.log(vr + epsilon) - ca.log(g))
    
    # The objective is to minimize the negative of the sum of these terms
    objective = -ca.sum1(forward_entropy + reverse_entropy)

    # 5. Define the Constraints
    # a) Steady-state constraint: S * v = 0, where v = vf - vr
    net_flux = vf - vr
    steady_state_constraint = ca.mtimes(S, net_flux)
    
    # b) Uptake constraint: v1 is fixed to 10
    uptake_constraint = net_flux[0] - 10
    
    # Combine all constraints into a single vector
    constraints = ca.vertcat(steady_state_constraint, uptake_constraint)

    # 6. Formulate the Nonlinear Programming (NLP) Problem
    nlp = {'x': x, 'f': -objective, 'g': constraints}

    # 7. Create a Solver
    # We use 'ipopt' for this nonlinear problem.
    solver_options = {'ipopt': {'print_level': 0, 'sb': 'yes'}, 'print_time': 0}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_options)

    # 8. Define Bounds and Initial Guess
    # Lower bounds for vf and vr are 0. Upper bounds are infinity.
    lbx = np.zeros(2 * num_reactions)
    ubx = np.full(2 * num_reactions, np.inf)
    
    # All constraints are equality constraints (equal to 0).
    lbg = np.zeros(num_metabolites + 1)
    ubg = np.zeros(num_metabolites + 1)
    
    # Initial guess for the fluxes [vf_1..4, vr_1..4]
    initial_guess = np.array([10.0, 10.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0])

    # 9. Solve the NLP Problem
    solution = solver(x0=initial_guess, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg)

    # 10. Extract and Print the Results
    if solver.stats()['success']:
        optimal_x = solution['x']
        optimal_vf = optimal_x[:num_reactions]
        optimal_vr = optimal_x[num_reactions:]
        optimal_v = optimal_vf - optimal_vr

        print("\n--- Entropy Maximization Results ---")
        print(f"Optimization was successful.")
        print(f"\nBiomass flux (v4): {optimal_v[3]}")
        
        print("\nOptimal Forward Fluxes (vf):")
        for i in range(num_reactions):
            print(f"  vf{i+1}: {optimal_vf[i]}")
            
        print("\nOptimal Reverse Fluxes (vr):")
        for i in range(num_reactions):
            print(f"  vr{i+1}: {optimal_vr[i]}")

        print("\nOptimal Net Fluxes (v = vf - vr):")
        for i in range(num_reactions):
            print(f"  v{i+1}: {optimal_v[i]}")
    else:
        print("\n--- Entropy Maximization Results ---")
        print("Optimization failed.")
        print("Solver statistics:", solver.stats())


if __name__ == "__main__":
    reversible_maxent()


Number of metabolites: 2
Number of reactions: 4

Stoichiometric Matrix (S):
[[ 1 -1  1  0]
 [ 0  1 -1 -1]]

--- Entropy Maximization Results ---
Optimization was successful.

Biomass flux (v4): 10

Optimal Forward Fluxes (vf):
  vf1: 11.2075
  vf2: 6.94787
  vf3: 1.94787
  vf4: 11.2075

Optimal Reverse Fluxes (vr):
  vr1: 1.20754
  vr2: 1.94787
  vr3: 6.94787
  vr4: 1.20754

Optimal Net Fluxes (v = vf - vr):
  v1: 10
  v2: 5
  v3: -5
  v4: 10


In [5]:
import numpy as np
import casadi as ca

def teraflux():
    """
    Performs a teraflux analysis based on maximizing a metabolic entropy function.

    This method splits each net flux v_i into forward (vf_i) and reverse (vr_i)
    components, where v_i = vf_i - vr_i.

    The network consists of two metabolites (A, B) and four reactions.
    The objective function to minimize is:
    -sum(vf_i * (log(vf_i) - log(g_i) ) + vr_i * (log(vr_i) - log(g_i) ) )

    This is equivalent to maximizing the entropy-like term.
    The problem is subject to steady-state constraints (S*v = 0), a
    fixed uptake flux for reaction 1 (v1 = 10), and a positive flux for v3.
    """
    # 1. Define the Stoichiometric Matrix (S)
    # Rows correspond to metabolites (A, B)
    # Columns correspond to reactions (v1, v2, v3, v4)
    S = np.array([
        [1, -1, 1, 0],   # Metabolite A
        [0, 1, -1, -1]    # Metabolite B
    ])
    
    # 2. Define model parameters
    num_metabolites, num_reactions = S.shape
    print(f"Number of metabolites: {num_metabolites}")
    print(f"Number of reactions: {num_reactions}")
    print("\nStoichiometric Matrix (S):")
    print(S)

    # 3. Set up the Optimization Problem using CasADi
    # We define separate variables for forward (vf) and reverse (vr) fluxes.
    vf = ca.SX.sym('vf', num_reactions)
    vr = ca.SX.sym('vr', num_reactions)
    
    # The full vector of optimization variables
    x = ca.vertcat(vf, vr)

    # 4. Define the Objective Function
    # Gene expression data (g)
    g = np.array([10, 10, 10, 10])
    
    # Small epsilon for numerical stability to avoid log(0)
    epsilon = 1e-6
    
    # Entropy terms for forward and reverse fluxes
    forward_entropy = vf * (ca.log(vf + epsilon) - ca.log(g))
    reverse_entropy = vr * (ca.log(vr + epsilon) - ca.log(g))
    
    # The objective is to minimize the negative of the sum of these terms
    objective = -ca.sum1(forward_entropy + reverse_entropy)

    # 5. Define the Constraints
    net_flux = vf - vr
    
    # a) Steady-state constraint: S * v = 0
    steady_state_constraint = ca.mtimes(S, net_flux)
    
    # b) Uptake constraint: v1 is fixed to 10
    uptake_constraint = net_flux[0] - 10
    
    # c) v3 positivity constraint: v3 >= 0
    v3_positivity_constraint = net_flux[2]
    
    # Combine all constraints into a single vector
    constraints = ca.vertcat(steady_state_constraint, uptake_constraint, v3_positivity_constraint)

    # 6. Formulate the Nonlinear Programming (NLP) Problem
    nlp = {'x': x, 'f': -objective, 'g': constraints}

    # 7. Create a Solver
    # We use 'ipopt' for this nonlinear problem.
    solver_options = {'ipopt': {'print_level': 0, 'sb': 'yes'}, 'print_time': 0}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_options)

    # 8. Define Bounds and Initial Guess
    # Lower bounds for vf and vr are 0. Upper bounds are infinity.
    lbx = np.zeros(2 * num_reactions)
    ubx = np.full(2 * num_reactions, ca.inf)
    
    # Define bounds for the constraints vector 'g'
    # The first (num_metabolites + 1) constraints are equality constraints (== 0)
    # The last constraint (v2 > 0) is an inequality constraint (>= 0)
    num_constraints = num_metabolites + 2
    lbg = np.zeros(num_constraints)
    ubg = np.zeros(num_constraints)
    ubg[-1] = ca.inf # Set upper bound for v3 constraint to infinity
    
    # Initial guess for the fluxes [vf_1..4, vr_1..4]
    initial_guess = np.array([10.0, 10.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0])

    # 9. Solve the NLP Problem
    solution = solver(x0=initial_guess, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg)

    # 10. Extract and Print the Results
    if solver.stats()['success']:
        optimal_x = solution['x']
        optimal_vf = optimal_x[:num_reactions]
        optimal_vr = optimal_x[num_reactions:]
        optimal_v = optimal_vf - optimal_vr

        print("\n--- Entropy Maximization Results ---")
        print(f"Optimization was successful.")
        print(f"\nBiomass flux (v4): {optimal_v[3]}")
        
        print("\nOptimal Forward Fluxes (vf):")
        for i in range(num_reactions):
            print(f"  vf{i+1}: {optimal_vf[i]}")
            
        print("\nOptimal Reverse Fluxes (vr):")
        for i in range(num_reactions):
            print(f"  vr{i+1}: {optimal_vr[i]}")

        print("\nOptimal Net Fluxes (v = vf - vr):")
        for i in range(num_reactions):
            print(f"  v{i+1}: {optimal_v[i]}")
    else:
        print("\n--- Entropy Maximization Results ---")
        print("Optimization failed.")
        print("Solver statistics:", solver.stats())


if __name__ == "__main__":
    teraflux()


Number of metabolites: 2
Number of reactions: 4

Stoichiometric Matrix (S):
[[ 1 -1  1  0]
 [ 0  1 -1 -1]]

--- Entropy Maximization Results ---
Optimization was successful.

Biomass flux (v4): 10

Optimal Forward Fluxes (vf):
  vf1: 11.2075
  vf2: 11.2075
  vf3: 3.67879
  vf4: 11.2075

Optimal Reverse Fluxes (vr):
  vr1: 1.20754
  vr2: 1.20754
  vr3: 3.67879
  vr4: 1.20754

Optimal Net Fluxes (v = vf - vr):
  v1: 10
  v2: 10
  v3: -7.75e-09
  v4: 10
